### Setup

In [1]:
import pandas as pd
import numpy as np
import os
from time import time

In [2]:
from vb_estimators import  LinRegSupreme,LinSVR,RBFSVR,ENet,L1Lars,GBR,HGBR,FlexiblePipe
from vb_helper import VBHelper
from vb_cross_validator import regressor_q_stratified_cv

no daal4py


In [3]:

from warnings import filterwarnings
filterwarnings('ignore')


### setup the experiment/project
#### note the 'run_stacked' kwarg that can be set to create the stacked_regressor

In [4]:
gridpoints=5
kwargs=dict(
    run_stacked=True,
    test_share=0,#keep at 0 for small datasets
    cv_folds=5,
    cv_reps=2,
    #cv_groupcount=5,
    cv_strategy=('quantile',5), # for stratified cv
    random_state=2 # random_state for reproducibility
)
vbhelper=VBHelper(**kwargs)

In [5]:
scorer_list=['neg_mean_squared_error', 'neg_mean_absolute_error', 'r2'] #cross_validate wants strings
vbhelper.scorer_list=scorer_list

##### User Import Dataset Step

In [6]:
data_path=os.path.join('sample_data','ex1.csv')
df=pd.read_csv(data_path)
all_vars=list(df.columns)
print(all_vars)

['STA_ID', 'LONG', 'LAT', 'OrigHabCode', 'Date', 'THG_Fish', 'YEAR', 'SEASON', 'SUBAREA', 'HABCODE', 'Floc_Depth_ft', 'AFDW_Floc', 'MEHG_Floc', 'THG_floc', 'Tot_Phos_floc', 'Bulk_Dens_Floc', 'Soil_Thickness_FT', 'AFDW_Soil', 'Bulk_Dens_Soil', 'PH_soil', 'SO4_soil', 'MEHG_soil', 'THG_soil', 'Tot_Carbon_Soil_%', 'Tot_Nitrogen_Soil_%', 'Tot_Phos_soil', 'Wat_Depth_ft', 'COND_SW', 'DO_SW', 'TEMP_SW', 'PH_SW', 'TURB_SW', 'REDOX_SW', 'Alk_Phos_SW', 'CHLA_SW', 'CL_SW', 'MEHG_SW', 'NH4_SW', 'NO2_SW', 'NO3_SW', 'SO4_SW', 'Sol_Reac_Phos_SW', 'THG_SW', 'TOC_SW', 'Tot_Nitrogen_SW', 'Tot_Phos_SW', 'REDOX_PW', 'H2S_PW', 'Sol_Reac_Phos_PW', 'MEHG_Peri_AVG', 'THG_epi_peri']


#### user has option to specify "regulatory standard"

In [7]:
#The user sets the variables to use for x and y.
y_name='THG_Fish'
loc_vars=['LAT','LONG']
drop_vars=['Date','OrigHabCode','STA_ID']
drop_vars.extend(loc_vars)
drop_vars.append(y_name)
x_vars=[var for var in all_vars if var not in drop_vars]
X_df=df.loc[10:,x_vars]
X_predict=df.loc[:10,x_vars]
y_df=df.loc[10:,y_name]
y_predict=df.loc[:10,y_name]


In [8]:

shuf=np.arange(y_df.shape[0])
seed=0
rng = np.random.default_rng(seed)
rng.shuffle(shuf)
X_df=X_df.iloc[shuf]
y_df=y_df.iloc[shuf]
vbhelper.setData(X_df,y_df)

### setup the analytical pipelines
#### note the inner_cv_dict and prep_dict that are used to consolidate vb_estimator kwargs and to facilitate the divison between prep and post steps.

In [9]:
inner_cv_dict={'cv_reps':1,'cv_folds':5,'cv_strategy':('quantile',5)}
inner_cv=vbhelper.getCV(cv_dict=inner_cv_dict)

prep_dict={'impute_strategy':'impute_knn5','cat_idx':vbhelper.cat_idx}

pipe_kwargs=dict(do_prep=not vbhelper.run_stacked,prep_dict=prep_dict,inner_cv=inner_cv,gridpoints=gridpoints,cat_idx=vbhelper.cat_idx,float_idx=vbhelper.float_idx,bestT=False)
pipe_dict={
    'gradient-boosting-reg':{'pipe':GBR,'pipe_kwargs':dict(
        prep_dict=prep_dict,do_prep=not vbhelper.run_stacked)},
    'rbf-svr-cv':{'pipe':RBFSVR,'pipe_kwargs':pipe_kwargs}, 
    'lassolars':{'pipe':L1Lars,'pipe_kwargs':pipe_kwargs},
    }
vbhelper.setPipeDict(pipe_dict) #formerly setEstimatorDict
vbhelper.setModelDict()


### end small test-run debugging

---

### Fit the final models for all estimators

In [10]:
vbhelper.runCrossValidate(try_load=True)

jhash:  57ae34d4efbc564326358b4e506514bd


In [11]:
vbhelper.refitPredictiveModels(selected_models=['multi_pipe'])

In [12]:
yhat=vbhelper.predict(X_predict)

In [13]:
yhat

{'multi_pipe': array([133.72272567, 235.93916116, 286.2431921 , 322.13360544,
        174.0426972 , 229.03323127, 144.79977695, 319.41767012,
        215.55609919, 239.838537  , 191.52909168]),
 'weights': {'multi_pipe': {'neg_mean_squared_error': 1.0,
   'neg_mean_absolute_error': 1.0,
   'r2': 1.0}},
 'prediction': {'neg_mean_squared_error': array([133.72272567, 235.93916116, 286.2431921 , 322.13360544,
         174.0426972 , 229.03323127, 144.79977695, 319.41767012,
         215.55609919, 239.838537  , 191.52909168]),
  'neg_mean_absolute_error': array([133.72272567, 235.93916116, 286.2431921 , 322.13360544,
         174.0426972 , 229.03323127, 144.79977695, 319.41767012,
         215.55609919, 239.838537  , 191.52909168]),
  'r2': array([133.72272567, 235.93916116, 286.2431921 , 322.13360544,
         174.0426972 , 229.03323127, 144.79977695, 319.41767012,
         215.55609919, 239.838537  , 191.52909168])}}

In [14]:
(yhat['prediction']['r2']-y_predict)/y_predict

0     0.844451
1     0.586679
2    -0.350628
3     0.281359
4     0.485006
5     2.413312
6    -0.235078
7    -0.413590
8     0.062900
9     0.361945
10    0.612198
Name: THG_Fish, dtype: float64

In [15]:
yhat_cv=vbhelper.predict(X_predict,model_type='cross_validation')

In [16]:
yhat_cv

{'multi_pipe': [array([113.77927668, 240.26033644, 280.80389021, 319.92397888,
         166.69500963, 209.42770141, 145.60401557, 332.95166873,
         217.05030752, 241.71733059, 184.92077113]),
  array([152.60042093, 272.63970255, 265.74513188, 306.52139527,
         203.21501611, 249.19849544, 169.54509056, 362.4662931 ,
         244.17515911, 245.18623872, 194.54152212]),
  array([163.22663582, 239.19396497, 264.62529948, 318.02913336,
         204.76985854, 239.82889626, 171.55699033, 341.24059627,
         231.98736932, 285.80815312, 213.70510065]),
  array([170.2447127 , 277.74179892, 298.35851042, 287.26028718,
         158.44323093, 236.71715522, 158.70270353, 333.24280506,
         223.09888838, 244.46751132, 198.21767112]),
  array([138.86468158, 229.25660745, 260.88860101, 277.12071428,
         180.69659203, 212.35001815, 141.97145851, 287.85791437,
         200.69220218, 250.21369323, 166.3899386 ]),
  array([160.83081113, 233.70161121, 281.39026919, 334.7232932 ,
      